In [ ]:
import torchvision
import torch
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from jetbot import Robot, Camera, bgr8_to_jpeg
from SCSCtrl import TTLServo
import time
import traitlets
from vision.detector import PlateNumberDetector
from vision.preprocessor import ImagePreprocessor
import cv2
import threading

# 모듈화한 파일들 import
from RobotMoving import RobotMoving
from WorkingAreaFind import WorkingAreaFind
from OCRTask import OCRTask

# 1. 전역 변수 및 설정
road_following_active = False  # 주행 활성화 플래그 (공용)
device = torch.device('cuda')

# 2. 하드웨어 초기화
robot = Robot()
camera = Camera.instance(width=816, height=616)
image = widgets.Image(format='jpeg', width=224, height=224)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

# 3. 로드 팔로잉 모델 로드
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model.load_state_dict(torch.load('best_steering_model_xy_test.pth')) # 모델 경로 확인
model = model.to(device).eval().half()

# 4. OCR 디텍터 및 서보 초기화 (객체가 이미 있다고 가정)
detector = PlateNumberDetector(
    model="clova",                 # OCR할 엔진 선택 ("paddle" 또는 "clova")
    plate_similarity_thresh=82,     # 타겟 번호판과 일치하는지 판단할 유사도 임계치(기본값 80)
    debug_mode=False,                # 디버그 모드 : 사전 OCR 데이터(data/demo/에 존재) 불러오기 -> API 호출 수 줄이거나 모델 로드 리소스 생략 가능
)


calib_data = np.load('fisheye_calib.npz')
K = calib_data['K']
D = calib_data['D']

WIDTH, HEIGHT = 224, 224

balance = 0.0 
new_K = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(
    K, D, (WIDTH, HEIGHT), np.eye(3), balance=balance
)
map1, map2 = cv2.fisheye.initUndistortRectifyMap(
    K, D, np.eye(3), new_K, (WIDTH, HEIGHT), cv2.CV_16SC2
)


In [ ]:
lbl1 = ipywidgets.Label(value="areaA :")


areaAlbl = ipywidgets.Label(value="None")
hbox1 = widgets.HBox([lbl1, areaAlbl] )

lbl2 = ipywidgets.Label(value="areaB :")
areaBlbl = ipywidgets.Label(value="None")
hbox2 = widgets.HBox([lbl2,areaBlbl])

lbl3 = ipywidgets.Label(value="self.flag :")
flaglbl = ipywidgets.Label(value="None")
hbox3 = widgets.HBox([lbl3, flaglbl])
vbox1 = widgets.VBox([hbox1, hbox2, hbox3])

image_widget = ipywidgets.Image(format='jpeg', width=224, height=224)
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')
vbox2 = widgets.VBox([image_widget, x_slider, steering_slider], layout=widgets.Layout(align_self='center'))
hbox4 = widgets.HBox([vbox2, y_slider, speed_slider], layout=widgets.Layout(align_self='center'))

startBtn = widgets.Button(description="Start", button_style='info')
lbl41 = ipywidgets.Label(value="Find Area : ")
goallbl = ipywidgets.Label(value="None")
hbox5 = widgets.HBox([startBtn,lbl41, goallbl])


#수동 조작용 contorller widget 생성
lbl50 = ipywidgets.Label(value="Manual Controller")

button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([lbl50, forward_button, middle_box, backward_button])

#자동 조작용 contorller widget 생성
lbl51 = ipywidgets.Label(value="Auto Controller")
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.25, description='speed gain',disabled = False)
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.1, description='steering gain',disabled = False)
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.25, description='steering kd',disabled = False)
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias',disabled = False)
vbox3 = widgets.VBox([lbl51,speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider])
hbox6 = widgets.HBox([controls_box, vbox3], layout=widgets.Layout(align_self='center'))

display(vbox1, hbox4,hbox5,hbox6)

manual_btnlst = [stop_button, forward_button, backward_button, left_button, right_button]
camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

flaglbl = widgets.Label(value='1')
goallbl = widgets.Label(value='Target: Area A')
areaAlbl = widgets.Label(value='Area A: Waiting')
areaBlbl = widgets.Label(value='Area B: Waiting')
ocr_label = widgets.Label(value='OCR Result: None')

# 위젯들을 클래스에 전달하기 위해 딕셔너리로 묶음
widgets_dict = {
    'image_widget': image_widget, 'x_slider': x_slider, 'y_slider': y_slider,
    'steering_slider': steering_slider, 'speed_slider': speed_slider,
    'speed_gain_slider': speed_gain_slider, 'steering_gain_slider': steering_gain_slider,
    'steering_dgain_slider': steering_dgain_slider, 'steering_bias_slider': steering_bias_slider,
    'flaglbl': flaglbl, 'goallbl': goallbl, 'areaAlbl': areaAlbl, 'areaBlbl': areaBlbl,
    'ocr_label': ocr_label
}

In [ ]:
def stop(change):
    robot.stop()
    
def step_forward(change):
    robot.forward(0.4)

def step_backward(change):
    robot.backward(0.4)

def step_left(change):
    robot.left(0.3)
    time.sleep(0.5)
    robot.stop()

def step_right(change):
    robot.right(0.3)
    time.sleep(0.5)
    robot.stop()

stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)

In [ ]:
from WorkingAreaFind import WorkingAreaFind

# 2. 색상 데이터 설정 (모듈화 전 코드 그대로 활용)
areaA_name = 'purple'
areaB_name = 'blue'

colors = [
    {'name': 'red', 'lower': np.array([0, 110, 155]), 'upper': np.array([6, 160, 222]) },
    {'name': 'green', 'lower': np.array([72, 75, 130]), 'upper': np.array([80, 150, 170]) },
    {'name': 'blue', 'lower': np.array([102, 155, 120]), 'upper': np.array([110, 252, 200]) },
    {'name': 'purple', 'lower': np.array([125, 60, 96]), 'upper': np.array([140, 120, 150]) },
    {'name': 'yellow', 'lower': np.array([23, 140, 170]), 'upper': np.array([32, 200, 210]) },
    {'name': 'orange', 'lower': np.array([14, 150, 150]), 'upper': np.array([20, 190, 200]) }
]

areaA_color = next((c for c in colors if c['name'] == areaA_name), None)
areaB_color = next((c for c in colors if c['name'] == areaB_name), None)

# 3. 위젯 업데이트 (초기값)
areaAlbl.value = areaA_color['name']
areaBlbl.value = areaB_color['name']
goallbl.value = "Target: " + areaA_color['name']



In [ ]:
ocr_worker = OCRTask(robot, camera, detector, TTLServo, widgets_dict, map1, map2)

# RobotMoving 생성
mover = RobotMoving(robot, camera, model, device, widgets_dict)

# WorkingAreaFind 생성 (ocr_worker를 인자로 넘겨서 도착 시 trigger 하도록 설정)
finder = WorkingAreaFind(camera, widgets_dict, ocr_task_obj=ocr_worker)

# 8. 시작/종료 버튼 로직
start_button = widgets.Button(description="Start AGV", button_style='success')
stop_button = widgets.Button(description="Stop AGV", button_style='danger')

# 쓰레드 시작
mover.start()
finder.start()
ocr_worker.start()